In [1]:
import pandas as pd
import numpy as np
import os
import scipy
from scipy.interpolate import griddata
import torch
import torch.nn as nn
from tqdm import tqdm
import torch.nn.functional as F
from torch.nn.functional import mse_loss
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from compute_distance import compute_distance

In [2]:
def original(x, y, d, n, ds = 0.4):
    """Compute the minimum distance of (x, y) from the nearest circular post."""
    tilt = ds/n
    centers = [(0, 0), (0, ds), (ds, tilt), (ds, ds+tilt)]
    
    distances = torch.full_like(x, float("inf"))  # Initialize distances with large values

    for cx, cy in centers:
        r = d / 2
        distance_to_post = torch.sqrt((x - cx) ** 2 + (y - cy) ** 2) - r
        distances = torch.minimum(distances, distance_to_post)  # Take the minimum distance

    return torch.maximum(distances, torch.tensor(0.0))  # Set negative distances to zero

In [3]:
# Folder path containing CSV files
folder_path = "data/unscaled"

# Define value ranges
first_values = np.arange(0.3, 0.7, 0.05)  # 0.25 to 0.7 with step 0.05
# second_values = np.arange(3, 15, 1)  # 3 to 14
second_values = np.arange(3, 15, 1)  # 3 to 14

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Iterate over all possible files
for first in first_values:
    for second in second_values:
        first_str = f"{first:.2f}".rstrip("0").rstrip(".")
        file_name = f"W_{first_str}_{second}_1.csv"
        file_path = os.path.join(folder_path, file_name)
        
        if os.path.exists(file_path):
            print(f"Processing {file_name}...")
            
            # Load CSV file
            df = pd.read_csv(file_path)
            
            # Extract input and output data
            input_data = df.iloc[:, [0, 1, 5, 7]].values  # (x, y, d, N) [0, 1, 5, 7]
            
            x = torch.tensor(input_data[:, 0], dtype=torch.float32, device=device).view(-1, 1)
            y = torch.tensor(input_data[:, 1], dtype=torch.float32, device=device).view(-1, 1)
            d = torch.tensor(input_data[:, 2], dtype=torch.float32, device=device).view(-1, 1)
            n = torch.tensor(input_data[:, 3], dtype=torch.float32, device=device).view(-1, 1)

            # Compute distance
            distances_1 = original(x, y, d, n)
            distances_2 = compute_distance(x, y, d, n)

            x_np = x.cpu().numpy().ravel()
            y_np = y.cpu().numpy().ravel()
            d1_np = distances_1.cpu().numpy().ravel()
            d2_np = distances_2.cpu().numpy().ravel()

            # plot the distances
            vmax = max(d1_np.max(), d2_np.max())

            plt.figure(figsize=(10, 5))
            plt.subplot(1, 2, 1)
            plt.scatter(x_np, y_np, c=d1_np, cmap='viridis', s=1)
            plt.colorbar(label='Distance')
            plt.title('Distance to Posts (original)')
            plt.xlabel('x')
            plt.ylabel('y')
            plt.subplot(1, 2, 2)
            plt.scatter(x_np, y_np, c=d2_np, cmap='viridis', s=1)
            plt.colorbar(label='Distance')
            plt.title('Distance to Posts (new)')
            plt.xlabel('x')
            plt.ylabel('y')
            plt.tight_layout()
            plt.savefig(f"results/distances/distances_{first_str}_{second}.png")
            plt.close()
            print(f"Saved distances plot for {file_name}")



Processing W_0.3_3_1.csv...
Saved distances plot for W_0.3_3_1.csv
Processing W_0.3_4_1.csv...
Saved distances plot for W_0.3_4_1.csv
Processing W_0.3_5_1.csv...
Saved distances plot for W_0.3_5_1.csv
Processing W_0.3_6_1.csv...
Saved distances plot for W_0.3_6_1.csv
Processing W_0.3_7_1.csv...
Saved distances plot for W_0.3_7_1.csv
Processing W_0.3_8_1.csv...
Saved distances plot for W_0.3_8_1.csv
Processing W_0.3_9_1.csv...
Saved distances plot for W_0.3_9_1.csv
Processing W_0.3_10_1.csv...
Saved distances plot for W_0.3_10_1.csv
Processing W_0.3_11_1.csv...
Saved distances plot for W_0.3_11_1.csv
Processing W_0.3_12_1.csv...
Saved distances plot for W_0.3_12_1.csv
Processing W_0.3_13_1.csv...
Saved distances plot for W_0.3_13_1.csv
Processing W_0.3_14_1.csv...
Saved distances plot for W_0.3_14_1.csv
Processing W_0.35_3_1.csv...
Saved distances plot for W_0.35_3_1.csv
Processing W_0.35_4_1.csv...
Saved distances plot for W_0.35_4_1.csv
Processing W_0.35_5_1.csv...
Saved distances plo